In [ ]:
!pip install openai-whisper
!pip install ultralytics
!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.4 MB/

In [ ]:
import pandas as pd
import numpy as np
import whisper
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# upload audio/video
from google.colab import files
uploaded = files.upload()

Saving 1minaudio.m4a to 1minaudio.m4a
Saving 1mindeskvideo.mp4 to 1mindeskvideo.mp4


In [ ]:
# whisper audio process
model = whisper.load_model("base")
result = model.transcribe("1minaudio.m4a")

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
# save transcript results
import csv

segments = result['segments']

with open("whisper_transcript.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["start", "end", "text"])
    for seg in segments:
        writer.writerow([seg['start'], seg['end'], seg['text']])

from google.colab import files
files.download("whisper_transcript.csv")

In [24]:
# yolo video process
model = YOLO("yolov8n.pt")  # 'n' = nano, fastest & smallest model
results = model("1mindeskvideo.mp4", save=True)  # save=True saves annotated video


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1835) /content/1mindeskvideo.mp4: 384x640 1 laptop, 149.4ms
video 1/1 (frame 2/1835) /content/1mindeskvideo.mp4: 384x640 1 laptop, 146.2ms
video 1/1 (frame 3/1835) /content/1mindeskvideo.mp4: 384x640 1 laptop, 139.5ms
video 1/1 (frame 4/1835) /content/1mindeskvideo.mp4: 384x640 1 suitcase, 1 laptop, 159.6ms
video 1/1 (frame 5/1835) /content/1mindeskvideo.mp4: 384x640 1 laptop, 155.0ms
video 1/1 (frame 6/1835) /content/1mindeskvideo.m

In [30]:
# save yolo results
with open("yolo_detections.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["frame_number", "object_class", "confidence_score", "x1", "y1", "x2", "y2"])

    for frame_idx, r in enumerate(results):
        boxes = r.boxes
        for box, conf, cls in zip(boxes.xyxy.cpu().numpy(),
                                  boxes.conf.cpu().numpy(),
                                  boxes.cls.cpu().numpy()):
            x1, y1, x2, y2 = box
            writer.writerow([frame_idx, int(cls), float(conf), x1, y1, x2, y2])

from google.colab import files
files.download('yolo_detections.csv')